<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/Mapa_de_Riesgos_Quito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análisis y modelado de vulnerabilidad ante riesgos naturales en Quito mediante Python, SIG y modelado espacial predictivo**

## Contenido
* [1. Introducción](#1-Introducción)
* [2. Importación de Librerías](#2-Importación-de-Librerías)
* [3. Carga y Descripción Inicial de Datos](#3-Carga-y-Descripción-Inicial-de-Datos)
* [4. Preprocesamiento y Transformación de Datos (ETL)](#4-Preprocesamiento-y-Transformación-de-Datos-(ETL)
  * [4.1 Conversión de TXT a CSV con Pandas](4.1-Conversión-de-TXT-a-CSV-con-Pandas)
  * [4.2 Limpieza de Datos](4.2-Limpieza-de-Datos)
  * [4.3 Ingeniería de Características (Feature Engineering)](4.3-Ingeniería-de-Características-(Feature-Engineering)
* [5. Integración de Datasets y Análisis Exploratorio Espacial](#5-Integración-de-Datasets-y-Análisis-Exploratorio-Espacial)
* [6. Modelado Predictivo de Riesgo](#6-Modelado-Predictivo-de-Riesgo)
* [7. Visualización Detallada de Resultados](#7-Visualización-Detallada-de-Resultados)
* [8. Conclusiones y Recomendaciones](#8-Conclusiones-y-Recomendaciones)
* [9. Referencias](#9-Referencias)

## 1. **Introducción**

La ciudad de Quito, capital de Ecuador, se encuentra en una zona geográfica compleja caracterizada por su relieve montañoso, su ubicación en el Cinturón de Fuego del Pacífico y su diversidad climática. Estas condiciones naturales hacen que la urbe esté expuesta a diferentes amenazas ambientales que ponen en riesgo a su población, infraestructura y ecosistemas urbanos.

Entre las amenazas más relevantes destacan:

- **Sismos:** Quito se ubica sobre una zona sísmicamente activa debido a la interacción de las placas tectónicas de Nazca y Sudamericana. Los movimientos telúricos representan uno de los mayores riesgos para la ciudad, especialmente en áreas con suelos blandos o edificaciones vulnerables.
- **Inundaciones:** Las precipitaciones intensas, combinadas con la expansión urbana y la deforestación de las laderas, incrementan la probabilidad de desbordamientos de quebradas y anegamientos en sectores bajos.
- **Olas de calor:** El cambio climático ha provocado un aumento de las temperaturas y una mayor frecuencia de eventos extremos, afectando la salud pública, la calidad del aire y el confort térmico en zonas densamente pobladas.

Frente a este contexto, resulta fundamental identificar las zonas más vulnerables de Quito mediante el uso de herramientas tecnológicas que integren información geográfica, estadística y ambiental. La programación en **Python** y los **Sistemas de Información Geográfica (SIG)** permiten analizar grandes volúmenes de datos espaciales y construir modelos predictivos que ayuden a la toma de decisiones basadas en evidencia.

Este proyecto busca aplicar técnicas de **análisis espacial y modelado predictivo** para generar un **mapa de vulnerabilidad ante riesgos naturales en Quito**, integrando diversas fuentes de datos abiertos.


### **Objetivo general:**
Analizar e identificar las zonas más vulnerables de Quito frente a amenazas naturales como sismos, inundaciones y olas de calor, utilizando herramientas de programación en Python, sistemas de información geográfica (SIG) y técnicas de modelado espacial para generar un mapa predictivo de riesgo y una visualización interactiva que facilite la interpretación de resultados.

### **Objetivos específicos:**
1. **Explorar y limpiar datasets abiertos** sobre peligros naturales (sísmicos, hidrológicos y climáticos) y características geográficas del Distrito Metropolitano de Quito.

2. **Integrar y georreferenciar capas de información** (geológica, hidrológica, topográfica, demográfica y climática) en un entorno de análisis geoespacial utilizando Python y librerías como `geopandas` y `rasterio`.

3. **Calcular indicadores de riesgo** combinando variables de amenaza, exposición y vulnerabilidad para cada zona o parroquia de Quito.

4. **Aplicar un modelo espacial predictivo** (por ejemplo, Random Forest espacial o Regresión logística geográfica) para estimar la probabilidad de riesgo en función de las características del terreno y los eventos históricos.

5. **Diseñar una visualización interactiva** de los resultados en Google Colab o mediante mapas dinámicos con `folium`o `plotly`, facilitando la interpretación de las áreas de mayor vulnerabilidad.

## 2. **Importación de Librerías**

In [4]:
# Importación de librerías principales
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")

## 3. **Carga y Descripción Inicial de Datos**

Los archivos que tienen datos sobre los sismos se encuentran en formato .txt lo que no es adecuado para el analisis. Se debe convertir a CSV

In [5]:
# Magnitud de los Sismos

columns_mag = [
    'event', 'orig_id', 'magnitude', 'magnitude_uncertainty',
    'magnitude_type', 'methodID', 'stationCount', 'agencyID'
]

archive_txt_mag = pd.read_csv(r'/content/cat_magnitud_2012-jul2025.txt',
                              sep='\,',
                              skiprows=27,
                              names=columns_mag,
                              encoding='utf-8')

print("--- 1. Magnitud de los Sismos (cat_magnitud) ---")
display(archive_txt_mag.head())

# Origen de los Sismos

columns_origen = ['event', 'orig_id', 'time_value', 'time_value_ms', 'time_uncertainty',
                  'latitude_value', 'latitude_uncertainty', 'longitude_value',
                  'longitude_uncertainty', 'depth_value', 'depth_uncertainty',
                  'magnitude_value_M', 'magnitude_value_P', 'magnitude_type_P',
                  'magnitudeP_uncertainty', 'magnitudeP_stationCount',
                  'quality_associatedPhaseCount', 'quality_usedPhaseCount',
                  'quality_associatedStationCount', 'quality_usedStationCount',
                  'quality_standardError', 'quality_azimuthalGap', 'quality_maximumDistance',
                  'quality_minimumDistance', 'quality_medianDistance', 'Fuente', 'methodID' ,
                  'earthModelID']
archive_txt_origen = pd.read_csv(r'/content/cat_origen_2012-jul2025.txt',
                                 sep='\,',
                                 skiprows=54,
                                 names=columns_origen,
                                 encoding='utf-8')

print("\n--- 2. Origen de los Sismos (cat_origen) ---")
display(archive_txt_origen.head())

# Tiempos de Onda de los Sismos

columns_picks = ['event', 'orig_id', 'time_value','time_value_ms','network_code','station_code',
                 'channel_code', 'phase_code', 'arri_azimuth', 'arri_distance', 'arri_timeResidual',
                 'arri_weight', 'polarity', 'time_used', 'agencyID'
]

archive_txt_picks = pd.read_csv(r'/content/cat_picks_2012-jul2025.txt',
                                sep='\,',
                                skiprows=33,
                                names=columns_picks,
                                encoding='utf-8')

print("\n--- 3. Tiempos de Onda de los Sismos (cat_picks) ---")
display(archive_txt_picks.head())


--- 1. Magnitud de los Sismos (cat_magnitud) ---


,event,orig_id,magnitude,magnitude_uncertainty,magnitude_type,methodID,stationCount,agencyID
0,igepn2012bafq,2766,3.6,0.33,MLv,mean,28,IG-EPN
1,igepn2012bafq,2766,3.6,NaN,M,weighted average,28,IG-EPN
2,igepn2012bbjo,2772,4.8,0.23,MLv,mean,14,IG-EPN
3,igepn2012bbjo,2772,4.8,NaN,M,weighted average,14,IG-EPN
4,igepn2012bhrl,2832,5.5,0.18,MLv,trimmed mean,27,IG-EPN



--- 2. Origen de los Sismos (cat_origen) ---


,event,orig_id,time_value,time_value_ms,time_uncertainty,latitude_value,latitude_uncertainty,longitude_value,longitude_uncertainty,depth_value,...,quality_associatedStationCount,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente,methodID,earthModelID
0,igepn2012acxo,2611,2012-01-02 14:10:00.000,507680,0.740,-1.291955,6.01,-80.529152,4.74,12.00,...,31,23,0.85,233.55,319.54,44.93,237.95,IGEPN,LOCSAT,iasp91
1,igepn2012ahkn,2643,2012-01-05 01:18:17.000,293170,1.268,-1.822281,11.44,-81.669304,9.91,5.00,...,14,14,1.35,322.97,458.75,285.82,376.04,IGEPN,LOCSAT,tab
2,igepn2012ahzg,2647,2012-01-05 08:43:48.000,885316,1.454,-1.651550,10.68,-81.512100,9.98,10.00,...,21,19,0.88,320.57,441.64,260.26,353.72,IGEPN,LOCSAT,tab
3,igepn2012aigm,2649,2012-01-05 12:22:12.000,310021,0.415,0.886300,3.04,-78.776039,1.81,1.17,...,28,24,0.78,175.91,265.47,47.15,153.12,IGEPN,LOCSAT,iasp91
4,igepn2012ajib,2651,2012-01-06 02:16:44.000,929684,1.041,-1.675611,13.42,-81.597900,7.58,10.00,...,14,12,0.45,334.43,392.87,328.99,351.19,IGEPN,LOCSAT,tab



--- 3. Tiempos de Onda de los Sismos (cat_picks) ---


,event,orig_id,time_value,time_value_ms,network_code,station_code,channel_code,phase_code,arri_azimuth,arri_distance,arri_timeResidual,arri_weight,polarity,time_used,agencyID
0,igepn2012acvi,2610,2012-01-02 13:03:15.000,123425,EC,MAG1,SHZ,P,40.48,210.01,-1.30,1.0,,NaN,IGEPN
1,igepn2012acvi,2610,2012-01-02 13:03:21.000,921816,EC,IGUA,SHZ,P,89.33,261.97,-1.00,1.0,,NaN,IGEPN
2,igepn2012acvi,2610,2012-01-02 13:03:23.000,3382,EC,ILLI,HHZ,P,70.86,266.67,-0.50,1.0,,NaN,IGEPN
3,igepn2012acvi,2610,2012-01-02 13:03:24.000,393961,EC,POND,HHZ,P,87.85,282.83,-1.12,1.0,,NaN,IGEPN
4,igepn2012acvi,2610,2012-01-02 13:03:25.000,681536,EC,RETU,SHZ,P,88.52,283.43,0.09,1.0,,NaN,IGEPN


## 4. **Preprocesamiento y Transformación de Datos (ETL)**

In [6]:
# Conteo de valores faltantes NaN
print("--- 1. Magnitud de los Sismos (cat_magnitud) ---")
print(archive_txt_mag.isnull().sum())

print("\n--- 2. Origen de los Sismos (cat_origen) ---")
print(archive_txt_origen.isnull().sum())

print("\n--- 3. Tiempos de Onda de los Sismos (cat_picks) ---")
print(archive_txt_picks.isnull().sum())

--- 1. Magnitud de los Sismos (cat_magnitud) ---
event                    0
orig_id                  0
magnitude                0
magnitude_uncertainty    0
magnitude_type           0
methodID                 0
stationCount             0
agencyID                 0
dtype: int64

--- 2. Origen de los Sismos (cat_origen) ---
event                             0
orig_id                           0
time_value                        0
time_value_ms                     0
time_uncertainty                  0
latitude_value                    0
latitude_uncertainty              0
longitude_value                   0
longitude_uncertainty             0
depth_value                       0
depth_uncertainty                 0
magnitude_value_M                 0
magnitude_value_P                 0
magnitude_type_P                  0
magnitudeP_uncertainty            0
magnitudeP_stationCount           0
quality_associatedPhaseCount      0
quality_usedPhaseCount            0
quality_associatedStationCou

In [7]:
# Magnitud del Sismo
# Limpiar incosistencias y forzar a valores númericos
archive_txt_mag['magnitude_uncertainty'] = pd.to_numeric(
    archive_txt_mag['magnitude_uncertainty'], errors='coerce'
)

# Eliminar valores duplicados
# Se mantiene el valor del primer registro
archive_txt_mag_consolidated = archive_txt_mag.drop_duplicates(
    subset=['event', 'orig_id'], # Identificador único del sismo
    keep='first'
).copy()

print("✅ Dataframe de Magnitud Limpio y Consolidado.")

display(archive_txt_mag_consolidated.head())

✅ Dataframe de Magnitud Limpio y Consolidado.


,event,orig_id,magnitude,magnitude_uncertainty,magnitude_type,methodID,stationCount,agencyID
0,igepn2012bafq,2766,3.6,0.33,MLv,mean,28,IG-EPN
2,igepn2012bbjo,2772,4.8,0.23,MLv,mean,14,IG-EPN
4,igepn2012bhrl,2832,5.5,0.18,MLv,trimmed mean,27,IG-EPN
11,igepn2012acvi,2610,3.8,0.34,MLv,trimmed mean,9,IG-EPN
13,igepn2012acxo,2611,3.8,0.76,MLv,mean,30,IG-EPN


In [8]:
# Origen del Sismo
# Conversión de Tipos de Datos (Crucial para geolocalización)
#  a) Fecha/Hora:
archive_txt_origen['time_value'] = pd.to_datetime(archive_txt_origen['time_value'], errors='coerce') #coerce sirve para reemplazar con NaN cualquier dato que no pueda convertirse

#  b) Numéricas: Las coordenadas y la profundidad tienen que ser flotantes
cols_to_numeric = [
    'time_value_ms', 'time_uncertainty', 'latitude_value', 'latitude_uncertainty',
    'longitude_value', 'longitude_uncertainty', 'depth_value', 'depth_uncertainty',
    'magnitude_value_M', 'magnitude_value_P', 'magnitudeP_uncertainty',
    'magnitudeP_stationCount', 'quality_standardError', 'quality_azimuthalGap',
    'quality_maximumDistance', 'quality_minimumDistance', 'quality_medianDistance'
]
for col in cols_to_numeric:
    archive_txt_origen[col] = pd.to_numeric(archive_txt_origen[col], errors='coerce')

# 2. Eliminación de Duplicados
archive_txt_origen_consolidated = archive_txt_origen.drop_duplicates(
    subset=['event', 'orig_id', 'time_value'],
    keep='first'
).copy()

# 3. Manejo de Nulos Esenciales
archive_txt_origen_consolidated.dropna(
    subset=['time_value', 'latitude_value', 'longitude_value', 'depth_value'],
    inplace=True
)

print("✅ Dataframe de Origen de Sismos Limpio y Consolidado.")
display(archive_txt_origen_consolidated.head())

✅ Dataframe de Origen de Sismos Limpio y Consolidado.


,event,orig_id,time_value,time_value_ms,time_uncertainty,latitude_value,latitude_uncertainty,longitude_value,longitude_uncertainty,depth_value,...,quality_associatedStationCount,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente,methodID,earthModelID
0,igepn2012acxo,2611,2012-01-02 14:10:00,507680,0.740,-1.291955,6.01,-80.529152,4.74,12.00,...,31,23,0.85,233.55,319.54,44.93,237.95,IGEPN,LOCSAT,iasp91
1,igepn2012ahkn,2643,2012-01-05 01:18:17,293170,1.268,-1.822281,11.44,-81.669304,9.91,5.00,...,14,14,1.35,322.97,458.75,285.82,376.04,IGEPN,LOCSAT,tab
2,igepn2012ahzg,2647,2012-01-05 08:43:48,885316,1.454,-1.651550,10.68,-81.512100,9.98,10.00,...,21,19,0.88,320.57,441.64,260.26,353.72,IGEPN,LOCSAT,tab
3,igepn2012aigm,2649,2012-01-05 12:22:12,310021,0.415,0.886300,3.04,-78.776039,1.81,1.17,...,28,24,0.78,175.91,265.47,47.15,153.12,IGEPN,LOCSAT,iasp91
4,igepn2012ajib,2651,2012-01-06 02:16:44,929684,1.041,-1.675611,13.42,-81.597900,7.58,10.00,...,14,12,0.45,334.43,392.87,328.99,351.19,IGEPN,LOCSAT,tab


In [14]:
# Tiempo de Onda del Sismo

# 1. Conversión de Tipos de Datos
# a) Fecha/Hora: Convertir la columna de tiempo.
archive_txt_picks['time_value'] = pd.to_datetime(
    archive_txt_picks['time_value'], errors='coerce'
)

# b) Numéricas: Forzamos a numérico
cols_to_numeric = [
    'time_value_ms', 'arri_azimuth', 'arri_distance', 'arri_timeResidual', 'arri_weight'
]

for col in cols_to_numeric:
    archive_txt_picks[col] = pd.to_numeric(archive_txt_picks[col], errors='coerce')

# 2. Manejo de Nulos Esenciales
archive_txt_picks_consolidated = archive_txt_picks.copy()
# Eliminamos filas si faltan la hora o la distancia de la onda
archive_txt_picks_consolidated.dropna(
    subset=['time_value', 'arri_distance'], inplace=True
)

print("✅ Dataframe de Tiempo de Onda de Sismos Limpio y Consolidado.")
display(archive_txt_picks_consolidated.head())

✅ Dataframe de Tiempo de Onda de Sismos Limpio y Consolidado.


,event,orig_id,time_value,time_value_ms,network_code,station_code,channel_code,phase_code,arri_azimuth,arri_distance,arri_timeResidual,arri_weight,polarity,time_used,agencyID
0,igepn2012acvi,2610,2012-01-02 13:03:15,123425,EC,MAG1,SHZ,P,40.48,210.01,-1.30,1.0,,NaN,IGEPN
1,igepn2012acvi,2610,2012-01-02 13:03:21,921816,EC,IGUA,SHZ,P,89.33,261.97,-1.00,1.0,,NaN,IGEPN
2,igepn2012acvi,2610,2012-01-02 13:03:23,3382,EC,ILLI,HHZ,P,70.86,266.67,-0.50,1.0,,NaN,IGEPN
3,igepn2012acvi,2610,2012-01-02 13:03:24,393961,EC,POND,HHZ,P,87.85,282.83,-1.12,1.0,,NaN,IGEPN
4,igepn2012acvi,2610,2012-01-02 13:03:25,681536,EC,RETU,SHZ,P,88.52,283.43,0.09,1.0,,NaN,IGEPN


**Nota:** En este caso de Tiempo de Onda de Sismos no eliminamos duplicados porque los datos son de diferentes intrumentos y estaciones por lo cual son necesarios para el análisis

## 4.1 **Conversión de TXT a CSV con Pandas**

## 4.2 **Limpieza de Datos**

## 4.3 **Ingeniería de Características (Feature Engineering)**

## 5. **Integración de Datasets y Análisis Exploratorio Espacial**

## 6. **Modelado Predictivo de Riesgo**

## 7. **Visualización Detallada de Resultados**

## 8. **Conclusiones y Recomendaciones**

## 9. **Referencias**